# 02_data_analysis
Dieses Notebook ist für die Auswertung unserer Daten

In [15]:

from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

project_root = Path('..').resolve()
output_dir = project_root / 'work/output'
processed_dir = output_dir / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

energy_zh = pd.read_csv(processed_dir / 'energy_zh.csv', parse_dates=['energyreporter_date'])
wealth_small = pd.read_csv(processed_dir / 'wealth_small.csv')

print('Loaded prepared inputs', energy_zh.shape, wealth_small.shape)


Loaded prepared inputs (9004, 5) (17176, 6)


In [16]:

# Energiekennzahlen 2024 aggregieren
energy_2024 = energy_zh[energy_zh['energyreporter_date'].dt.year == 2024].copy()
consum = (
    energy_2024
    .groupby('bfs_nr', as_index=False)
    .agg({
        'elec_consumption_mwh_per_year_per_capita': 'mean',
        'elec_consumption_households_mwh_per_year_per_capita': 'mean'
    })
    .rename(columns={
        'elec_consumption_mwh_per_year_per_capita': 'total_elec_per_capita_mwh_2024',
        'elec_consumption_households_mwh_per_year_per_capita': 'household_elec_per_capita_mwh_2024'
    })
)

consum.to_csv(processed_dir / 'energy_consumption_2024.csv', index=False)
consum.head()


,bfs_nr,total_elec_per_capita_mwh_2024,household_elec_per_capita_mwh_2024
0,1,4.495275,2.970700
1,2,4.198583,1.678750
2,3,2.789025,1.738283
3,4,3.874125,2.251017
4,5,5.465508,2.169908


In [17]:
# Vermögenskennzahlen 2022 extrahieren
wealth_2022 = wealth_small[wealth_small['INDIKATOR_JAHR'] == 2022].copy()
patterns = {
    'income_median_2022': r'^Steuerb\. Einkommen.*Median',
    'income_avg_2022': r'^Steuerb\. Einkommen.*Durchschn',
    'income_total_per_capita_2022': r'^Steuerb\. Einkommen.*(je Einwohner|pro Kopf)',
    'capital_median_2022': r'^Steuerb\. Vermögen.*Median',
    'capital_avg_2022': r'^Steuerb\. Vermögen.*Durchschn',
    'capital_total_per_capita_2022': r'^Steuerb\. Vermögen.*(je Einwohner|pro Kopf)'
}
wealth_2022['metric'] = np.nan
for col, pattern in patterns.items():
    match_mask = wealth_2022['INDIKATOR_NAME'].str.contains(pattern, regex=True, na=False)
    wealth_2022.loc[match_mask, 'metric'] = col

wide = (
    wealth_2022.dropna(subset=['metric'])
    .pivot_table(index='BFS_NR', columns='metric', values='INDIKATOR_VALUE', aggfunc='first')
    .reset_index()
    .rename(columns={'BFS_NR': 'bfs_nr'})
)
wide.to_csv(processed_dir / 'wealth_metrics_2022.csv', index=False)
wide.head()

/tmp/ipykernel_334/3472260061.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'income_median_2022' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  wealth_2022.loc[match_mask, 'metric'] = col
/tmp/ipykernel_334/3472260061.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match_mask = wealth_2022['INDIKATOR_NAME'].str.contains(pattern, regex=True, na=False)
/tmp/ipykernel_334/3472260061.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match_mask = wealth_2022['INDIKATOR_NAME'].str.contains(pattern, regex=True, na=False)


metric,bfs_nr,capital_avg_2022,capital_median_2022,income_avg_2022,income_median_2022
0,0,491000.0,99000.0,78800.0,60100.0
1,1,851000.0,200000.0,96700.0,68800.0
2,2,334000.0,48000.0,62900.0,52400.0
3,3,432000.0,118000.0,80400.0,67100.0
4,4,523000.0,125000.0,78800.0,57200.0


In [18]:

# Energie- und Vermögensdaten kombinieren
municipality_lookup = energy_zh[['bfs_nr', 'municipality']].drop_duplicates(subset='bfs_nr')
merged = consum.merge(wide, on='bfs_nr', how='inner')
merged = merged.merge(municipality_lookup, on='bfs_nr', how='left')

# Fallback für Gemeindebezeichnungen aus der Vermögensdatei
wealth_names = wealth_small[['BFS_NR', 'GEBIET_NAME']].drop_duplicates(subset='BFS_NR')
wealth_names = wealth_names.rename(columns={'BFS_NR': 'bfs_nr', 'GEBIET_NAME': 'municipality_wealth'})
merged = merged.merge(wealth_names, on='bfs_nr', how='left')
if 'municipality' not in merged.columns:
    merged['municipality'] = merged['municipality_wealth']
else:
    merged['municipality'] = merged['municipality'].fillna(merged['municipality_wealth'])
merged = merged.drop(columns=['municipality_wealth'])

# Numerische Spalten bereinigen und Verhältnisse bilden
value_cols = [
    'income_median_2022','income_avg_2022','income_total_per_capita_2022',
    'capital_median_2022','capital_avg_2022','capital_total_per_capita_2022'
]
for col in value_cols:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors='coerce').replace(0, np.nan)

if 'capital_median_2022' in merged.columns:
    merged['ratio_consum_to_capital_median'] = merged['household_elec_per_capita_mwh_2024'] / merged['capital_median_2022']
if 'capital_avg_2022' in merged.columns:
    merged['ratio_consum_to_capital_avg'] = merged['household_elec_per_capita_mwh_2024'] / merged['capital_avg_2022']
if 'capital_total_per_capita_2022' in merged.columns:
    merged['ratio_consum_to_capital_total'] = merged['household_elec_per_capita_mwh_2024'] / merged['capital_total_per_capita_2022']

merged.to_csv(processed_dir / 'merged_energy_wealth.csv', index=False)
print(f"Merged dataset saved to {processed_dir / 'merged_energy_wealth.csv'}")
merged.head()


Merged dataset saved to /home/jovyan/work/output/processed/merged_energy_wealth.csv


,bfs_nr,total_elec_per_capita_mwh_2024,household_elec_per_capita_mwh_2024,capital_avg_2022,capital_median_2022,income_avg_2022,income_median_2022,municipality,ratio_consum_to_capital_median,ratio_consum_to_capital_avg
0,1,4.495275,2.970700,851000.0,200000.0,96700.0,68800.0,Aeugst am Albis,0.000015,0.000003
1,2,4.198583,1.678750,334000.0,48000.0,62900.0,52400.0,Affoltern am Albis,0.000035,0.000005
2,3,2.789025,1.738283,432000.0,118000.0,80400.0,67100.0,Bonstetten,0.000015,0.000004
3,4,3.874125,2.251017,523000.0,125000.0,78800.0,57200.0,Hausen am Albis,0.000018,0.000004
4,5,5.465508,2.169908,556000.0,111000.0,83100.0,62800.0,Hedingen,0.000020,0.000004


In [19]:
merged = pd.read_csv(processed_dir / 'merged_energy_wealth.csv')
print('Merged dataset loaded:', merged.shape)

Merged dataset loaded: (160, 10)


In [20]:

# Einfache Korrelationsanalyse für Haushaltsstrom
corr_rows = []
wealth_cols = [c for c in [
    'income_median_2022','income_avg_2022','income_total_per_capita_2022',
    'capital_median_2022','capital_avg_2022','capital_total_per_capita_2022'
] if c in merged.columns]
for col in wealth_cols:
    df = merged[['household_elec_per_capita_mwh_2024', col]].dropna()
    if df.empty:
        continue
    pr, pp = pearsonr(df['household_elec_per_capita_mwh_2024'], df[col])
    sr, sp = spearmanr(df['household_elec_per_capita_mwh_2024'], df[col])
    corr_rows.append({'metric': col, 'pearson_r': pr, 'pearson_p': pp, 'spearman_r': sr, 'spearman_p': sp, 'n': len(df)})

corr_df = pd.DataFrame(corr_rows).sort_values('pearson_r', ascending=False)
corr_df.to_csv(output_dir / 'correlations.csv', index=False)
corr_df

,metric,pearson_r,pearson_p,spearman_r,spearman_p,n
2,capital_median_2022,0.429146,1.494696e-08,0.546143,8.049780e-14,160
0,income_median_2022,0.072170,3.644546e-01,0.159037,4.456785e-02,160
3,capital_avg_2022,0.000395,9.960462e-01,0.240025,2.234785e-03,160
1,income_avg_2022,-0.053187,5.041569e-01,0.020347,7.984344e-01,160


In [21]:

# Umfassende Korrelationsmatrix für beide Energievarianten
energy_cols = ['household_elec_per_capita_mwh_2024', 'total_elec_per_capita_mwh_2024']
all_corr_rows = []
for energy_col in energy_cols:
    if energy_col not in merged.columns:
        continue
    for wealth_col in wealth_cols:
        if wealth_col not in merged.columns:
            continue
        df = merged[[energy_col, wealth_col]].dropna()
        if df.empty:
            continue
        pr, pp = pearsonr(df[energy_col], df[wealth_col])
        sr, sp = spearmanr(df[energy_col], df[wealth_col])
        all_corr_rows.append({
            'energy_metric': energy_col,
            'wealth_metric': wealth_col,
            'pearson_r': pr,
            'pearson_p': pp,
            'spearman_r': sr,
            'spearman_p': sp,
            'n': len(df)
        })

all_corr_df = pd.DataFrame(all_corr_rows).sort_values('pearson_r', key=abs, ascending=False)
all_corr_df.to_csv(output_dir / 'comprehensive_correlations.csv', index=False)
all_corr_df

,energy_metric,wealth_metric,pearson_r,pearson_p,spearman_r,spearman_p,n
2,household_elec_per_capita_mwh_2024,capital_median_2022,0.429146,1.494696e-08,0.546143,8.049780e-14,160
4,total_elec_per_capita_mwh_2024,income_median_2022,-0.194092,1.392165e-02,-0.231301,3.252351e-03,160
5,total_elec_per_capita_mwh_2024,income_avg_2022,-0.181915,2.132035e-02,-0.257328,1.020013e-03,160
6,total_elec_per_capita_mwh_2024,capital_median_2022,-0.126781,1.101429e-01,-0.086161,2.786609e-01,160
7,total_elec_per_capita_mwh_2024,capital_avg_2022,-0.112738,1.557883e-01,-0.157899,4.613532e-02,160
0,household_elec_per_capita_mwh_2024,income_median_2022,0.072170,3.644546e-01,0.159037,4.456785e-02,160
1,household_elec_per_capita_mwh_2024,income_avg_2022,-0.053187,5.041569e-01,0.020347,7.984344e-01,160
3,household_elec_per_capita_mwh_2024,capital_avg_2022,0.000395,9.960462e-01,0.240025,2.234785e-03,160


In [22]:

# Ausreissererkennung
outlier_rows = []
features = ['household_elec_per_capita_mwh_2024', 'total_elec_per_capita_mwh_2024']
model = LinearRegression()

for col in wealth_cols:
    cols_available = [f for f in features if f in merged.columns]
    if not cols_available:
        continue
    df = merged[['bfs_nr', col] + cols_available].dropna()
    if df.empty:
        continue
    X = df[cols_available]
    y = df[col]
    model.fit(X, y)
    y_pred = model.predict(X)
    resid = y - y_pred
    scaler = StandardScaler()
    z = scaler.fit_transform(resid.to_frame(name='residual')).flatten()
    outlier_rows.append(pd.DataFrame({'bfs_nr': df['bfs_nr'].values, 'metric': col, 'residual': resid, 'zscore': z}))

outliers = pd.concat(outlier_rows, ignore_index=True) if outlier_rows else pd.DataFrame(columns=['bfs_nr','metric','residual','zscore'])
outliers_sorted = outliers.reindex(outliers['zscore'].abs().sort_values(ascending=False).index)
outliers.to_csv(output_dir / 'outliers.csv', index=False)
outliers_sorted.head(10)


,bfs_nr,metric,residual,zscore
572,139,capital_avg_2022,3.900678e+06,6.235607
574,151,capital_avg_2022,3.854703e+06,6.162113
577,154,capital_avg_2022,2.956246e+06,4.725842
252,139,income_avg_2022,1.030577e+05,4.188440
248,135,income_avg_2022,1.023003e+05,4.157657
254,151,income_avg_2022,1.021730e+05,4.152485
88,135,income_median_2022,3.003891e+04,4.087311
147,248,income_median_2022,2.965345e+04,4.034863
467,248,capital_median_2022,1.796922e+05,3.801622
415,152,capital_median_2022,1.790981e+05,3.789051
